Giulia Rodrigues
Paradigmas de Programação - 02/2023 - UFSM-RS 

In [ ]:
import pandas as pd

In [ ]:
FLIGHTS_PATH = './flights.csv'
SEATS_PATH = './seats.csv'

In [ ]:
flights_df = pd.read_csv(FLIGHTS_PATH)
seats_df = pd.read_csv(SEATS_PATH)

In [ ]:
seats_per_plane_size = {
    'S': 4,
    'M': 6,
    'L': 9
}

rows_per_plane_size = {
    'S': 'H',
    'M': 'P',
    'L': 'Z'
}

In [ ]:
def does_flight_exists(flight_number) :
    return len(flights_df.loc[flights_df['flight_number'] == flight_number]) > 0

def get_seats_from_flight(flight_number) :
    return seats_df.loc[seats_df['flight_number'] == flight_number]

def get_plane_size(flight_number) :
    return flights_df.loc[flights_df['flight_number'] == flight_number]['plane_size'].iloc[0]

def is_seat_occupied(flight_number, seat_number) :
    return len(seats_df.loc[(seats_df['flight_number'] == flight_number) & (seats_df['seat_number'] == seat_number)]) > 0

def is_seat_valid(seat_number, plane_size) :
    max_seats = seats_per_plane_size[plane_size]
    
    if len(seat_number) != 2 :
        return False
    
    if seat_number[0].isdigit() :
        return False
    
    if not seat_number[1].isdigit() :
        return False
    
    if int(seat_number[1]) > max_seats :
        return False
    
    return True

def get_filled_flight_seats_dict(flight_seats, plane_size) :
    filled_flight_seats_dict = {}
    
    rows_range = [chr(i) for i in range(ord('A'), ord(rows_per_plane_size[plane_size]) + 1)]
    
    for row in rows_range :
        filled_flight_seats_dict[row] = []
    
    if not flight_seats.empty:
        for index, row in flight_seats.iterrows():
            seat = row['seat_number']
            letter = seat[0]
            filled_flight_seats_dict[letter].append((seat, False))

    for row in filled_flight_seats_dict :
        columns = [number[1] for number, _ in filled_flight_seats_dict[row]]
        for i in range(1, seats_per_plane_size[plane_size] + 1) :
            if str(i) not in columns :
                filled_flight_seats_dict[row].append((row + str(i), True))

        filled_flight_seats_dict[row] = sorted(filled_flight_seats_dict[row])

    return filled_flight_seats_dict

def buy_seat(flight_number, seat_number, passenger_name, save = True) :
    if not does_flight_exists(flight_number) :
        print('O voo não existe!')
        return
        
    if is_seat_occupied(flight_number, seat_number) :
        print('O assento já foi comprado!')
        return
    
    seats_df.loc[len(seats_df)] = [flight_number, seat_number, passenger_name]
    
    if save :
        seats_df.to_csv(SEATS_PATH, index=False)
        print('Assento comprado com sucesso!')
        
    return       

def display_seats(flight_number) :
    step_by_plane_size = {
        'S': 2,
        'M': 3,
        'L': 3
    }
    
    flight_seats = get_seats_from_flight(flight_number)[['seat_number']]

    plane_size = get_plane_size(flight_number)
    filled_flight_seats_dict = get_filled_flight_seats_dict(flight_seats, plane_size)
    
    step = step_by_plane_size[plane_size]
    
    row_label_distance = 2
    corridor_distance = 5
    
    for row in filled_flight_seats_dict :
        i = 0
        row_str = '\033[97m' + row + row_label_distance * ' '
        
        for number, available in filled_flight_seats_dict[row] :
            if i == step :
                row_str += corridor_distance * ' '
                i = 0
                
            if available :
                row_str += '\033[92m'
            else :
                row_str += '\033[91m'
                
            row_str += number + ' '
            
            i += 1
        
        row_str += '\033[97m' + (row_label_distance - 1) * ' ' + row
        
        print(row_str)
        
def is_flight_full(flight_number):
    seats_per_row = seats_per_plane_size[get_plane_size(flight_number)]

    flight_seats = get_seats_from_flight(flight_number)['seat_number']
    total_seats = len(flight_seats)
    max_seats = seats_per_row * (ord(rows_per_plane_size[get_plane_size(flight_number)]) - ord('A') + 1)
    
    return total_seats >= max_seats

def get_flight_info(flight_number):
    if does_flight_exists(flight_number):
        plane_size = get_plane_size(flight_number)
        origin = flights_df.loc[flights_df['flight_number'] == flight_number]['origin'].iloc[0]
        destination = flights_df.loc[flights_df['flight_number'] == flight_number]['destination'].iloc[0]
        departure_date = flights_df.loc[flights_df['flight_number'] == flight_number]['departure_date'].iloc[0]
        
        return {
            'flight_number': flight_number,
            'plane_size': plane_size,
            'origin': origin,
            'destination': destination,
            'departure': departure_date
        }
    else:
        return None

def create_flight(flight_number, plane_size, origin, destination, departure_date, arrival_date, available):
    if does_flight_exists(flight_number):
        return "O voo já existe."
    
    if plane_size not in seats_per_plane_size:
        return "Tamanho inválido, escolha entre S, M ou L."
    
    flights_df.loc[len(flights_df)] = [flight_number, plane_size, origin, destination, departure_date, arrival_date, available]
    flights_df.to_csv(FLIGHTS_PATH, index=False)
    
    return "Voo criado com sucesso."

In [ ]:
get_flight_info('L800')

In [ ]:
display_seats('L800')
buy_seat('L800', 'G4', 'Gabriele')
display_seats('L800')

In [ ]:
is_seat_occupied('L800', 'G4')

In [ ]:
get_seats_from_flight('L800')

In [ ]:
display_seats('S100')
is_flight_full('S100')

In [ ]:
is_flight_full('L800')

In [ ]:
create_flight('S200', 'S', 'Sydney', 'Tokyo', '2023-12-03T15:00:00','2023-12-03T23:00:00',True)

In [ ]:
create_flight('M342', 'M', 'Florianópolis', 'Rio de Janeiro', '2023-12-03T23:00:00','2023-12-04T12:00:00',True)

### Conclusões:

Mesmo que o trabalho não tenha seguido a proposta original que combinamos, eu escolhi fazer o trabalho em python e não me arrependi dessa escolha. Eu estou vendo python em deep learning nesse semestre e esse trabalho me ajudou a realmente colocar as coisas na prática, entender melhor a linguagem e a me familiarizar com ela. No começo eu tinha bastante preconceito com python porque já escutei que *"meninas só sabem programar usando isso porque é mais fácil"*, mas finalmente entendi o apelo da linguagem e porque ela é tão procurada.

Eu escolhi trabalhar com a biblioteca pandas porquê: primeiro, o nome parecia interessante, mas, não tem nada a ver com o animalzinho, segundo, descobri que trabalha com dataframes e possui um monte de bibliotecas auxiliares então daria para fazer um trabalho legal com ela.

A biblioteca permite fazer a leitura e escrita com vários tipos de arquivos, e nesse trabalho tentei explorar o XML e o csv. 

O XML começou a ficar extremamente maçante e eu não consegui fazer muita coisa com ele, então, parti para o csv, que foi bem mais fácil de trabalhar. 

Depois que aprendi a manusear um pouco arquivos csv, comecei a pesquisar possíveis funções que eu poderia chamar para resolver questões no código.

Os métodos loc, iloc e isdigit foram os principais métodos que eu utilizei para resolver as questões do trabalho.
Quando o método loc se tornou familiar, consegui montar as lógicas das funções com mais facilidade pois ajuda a localizar os dados que eu preciso facilmente.

Alguns outros métodos já eram familiares pois já tinha visto versões deles em haskell, como o head, tail, length, etc.
Já outros métodos foram novos para mim, como o isdigit e oS conceitos de nan/none e dicionários, foi assustador no começo, mas de acordo com o que eu ia descobrindo, fui me empolgando em achar metódos novos e jeitos diferentes de criar minhas funções.
 
A função to_csv também foi muito importante, pois foi através dela que consegui fazer o buy_seats funcionar e atualizar a tabela de assentos.

Foi um trabalho bem divertido de fazer, apesar de ter mudado toda a proposta de ultima hora e ter enviado super atrasado, sinto que na verdade fiz um progresso imenso, quando voltei para a faculdade no ínicio desse ano, eu já não sabia se iria dar conta de programar mesmo, entrei em 2018 e fui trancando durante vários semestres até me organizar, tomei gosto de fazer código quando comecei a fazer ED e lab2, isso só se fortaleceu até agora em paradigmas, é extremamente prazeroso conseguir criar algo e fico muito feliz de ter espaço para meninas nessa área também. :)

Espero que java seja tão divertido quanto python, estou ansiosa para aprender mais sobre programaçao orientada a objetos <3 

### Dificuldades:

Acabei tendo alguns travamentos em certas funções:
1. Perdi um tempo por uma besteira na função create_flight(), eu estava enviando 7 argumentos para a função, mas ela só esperava 5, até eu notar que esse era o problema, eu já tinha perdido um tempo considerável.

2. Na função get_filled_flight_seats_dict(), display_seats() e get_flight_full foi aonde eu mais senti dificuldade, tive que procurar ajuda de colegas que já passaram pela disciplina para elaborar elas, apesar de ter conseguido pensar sozinha na lógica de tudo, ocorreu o problema de não saber o que usar para o código fazer o que eu queria, então, tive que pedir ajuda.

3. Na função buy_seats() eu tive que pesquisar bastante para entender como funcionava o método to_csv, pois eu não estava conseguindo fazer a atualização da tabela de assentos, mas depois de pesquisar e entender como funcionava, consegui fazer a função funcionar.

4. A questão da coloração de assentos foi algo totalmente novo para mim, nem sabia por onde começar mas percebi que era muito tranquilo, o trabalho acabou me empurrando para testar coisas novas e divertidas também.


### Referências:
+ [Por quê usar a biblioteca pandas?](https://harve.com.br/blog/programacao-python-blog/pandas-python-vantagens-e-como-comecar/)
+ [Como é e como instalar a biblioteca](https://www.alura.com.br/artigos/pandas-o-que-e-para-que-serve-como-instalar)
+ [Como trabalhar com arquivos csv](https://www.w3schools.com/python/pandas/pandas_csv.asp)
+ [Diferentes formas de fazer a leitura de um arquivo csv com pandas](https://medium.com/@henrique.gelatti/diferentes-formas-de-ler-um-arquivo-csv-utilizando-a-biblioteca-pandas-dbeab96555ba)
+ [Como usar os métodos loc e iloc](https://medium.com/horadecodar/data-science-tips-02-como-usar-loc-e-iloc-no-pandas-fab58e214d87)
+ [loc vs iloc](https://www.delftstack.com/pt/howto/python-pandas/pandas-loc-vs-iloc-python/)
+ [Como usar o método isdigit()](https://acervolima.com/python-pandas-series-str-isdigit/)
+ [O quê é nan/none](https://note.nkmk.me/en/python-pandas-nan-none-na/)
+ [Como funciona um dicionário em python](https://blog.somostera.com/desenvolvimento-web/dicionario-python)
+ [Método to_csv](https://www.aprendadatascience.com/blog/fun%C3%A7%C3%A3o-to_csv)